# Introduction

Here we aim to classify images into daylight landscapes, nightlight landscapes and portraits by designing features and using kNN classifier.

In [176]:
# all imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.decomposition import KernelPCA

In [177]:
# get minimum dimensions
def get_min_dimensions():
    MIN_WIDTH = 500
    MIN_HEIGHT = 500
    for class_folder in ['portrait', 'day-landscape', 'night-landscape']:        
        for file in os.listdir('../Train Images/' + class_folder):
            img = cv2.imread('../Train Images/' + class_folder + '/' + file)
            if (img.shape[0] < MIN_HEIGHT):
                MIN_HEIGHT = img.shape[0]
            if (img.shape[1] < MIN_WIDTH):
                MIN_WIDTH = img.shape[1]
    
    for file in os.listdir('../Test Images/'):
        img = cv2.imread('../Test Images/' + file)
        if (img.shape[0] < MIN_HEIGHT):
            MIN_HEIGHT = img.shape[0]
        if (img.shape[1] < MIN_WIDTH):
            MIN_WIDTH = img.shape[1]
        
    return (MIN_HEIGHT, MIN_WIDTH)

In [178]:
# resize images
def resize_images(img, min_dims):
    resized_img = cv2.resize(img, min_dims, cv2.INTER_AREA)
    return resized_img

In [179]:
# get mean value
def get_mean_value(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mean_value = np.mean(hsv_img[:,:,2])
    return hsv_img, mean_value

In [180]:
# apply sobel filter
def apply_sobel(img):
    blur_img = cv2.GaussianBlur(img, (3, 3), 0)
    gray_img = cv2.cvtColor(blur_img, cv2.COLOR_BGR2GRAY)
    grad_y = cv2.Sobel(gray_img, cv2.CV_16S, 1, 0, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
    abs_gray_y = cv2.convertScaleAbs(grad_y)
    return abs_gray_y

In [181]:
# image preprocessing
min_dims = get_min_dimensions()
print(min_dims)
for class_folder in ['portrait', 'day-landscape', 'night-landscape']:        
    for file in os.listdir('../Train Images/' + class_folder):
#         print(file)
        img = cv2.imread('../Train Images/' + class_folder + '/' + file)
        resize_image = resize_images(img, min_dims)
        plt.imsave('./Resized Images/' + class_folder + '/' + file, resize_image)

for file in os.listdir('../Test Images/'):
#     print(file)
    img = cv2.imread('../Test Images/' + file)
    resize_image = resize_images(img, min_dims)
    plt.imsave('./Resized Images/Test Images/' + file, resize_image)

(138, 182)


In [182]:
# class dictionary
class_dict = {
    'portrait': 0,
    'day-landscape': 1,
    'night-landscape': 2
}

# get image features
train_feats = []
labels = []

for class_folder in ['portrait', 'day-landscape', 'night-landscape']:
    for file in os.listdir('./Resized Images/' + class_folder):
        img = cv2.imread('./Resized Images/' + class_folder + '/' + file)
        hsv_img, mean_value = get_mean_value(img)
        plt.imsave('./HSV Images/' + class_folder + '/' + file, hsv_img)
        abs_gray_y = apply_sobel(img)
        plt.imsave('./Sobel Responses/' + class_folder + '/' + file, abs_gray_y, cmap='gray')
        abs_gray_y_flatten = abs_gray_y.flatten()
        abs_gray_y_flatten_list = abs_gray_y_flatten.tolist()
        abs_gray_y_flatten_list.append(mean_value)
        feat_vec = np.array(abs_gray_y_flatten_list)
        train_feats.append(feat_vec)
        labels.append(class_folder)        

train_feats = np.array(train_feats, dtype=np.float32)


labels = list(map(lambda class_label : class_dict.get(class_label), labels))
labels = np.array(labels, dtype=np.float32)
print(train_feats.shape, labels.shape)

(42, 25117) (42,)


In [183]:
# get test features
test_feats = []
for file in os.listdir('./Resized Images/Test Images'):
    img = cv2.imread('./Resized Images/Test Images/' + file)
    hsv_img, mean_value = get_mean_value(img)
    plt.imsave('./HSV Images/Test Images/' + file, hsv_img)
    abs_gray_y = apply_sobel(img)
    plt.imsave('./Sobel Responses/Test Images/' + file, abs_gray_y, cmap='gray')
    abs_gray_y_flatten = abs_gray_y.flatten()
    abs_gray_y_flatten_list = abs_gray_y_flatten.tolist()
    abs_gray_y_flatten_list.append(mean_value)
    feat_vec = np.array(abs_gray_y_flatten_list)
    test_feats.append(feat_vec)
    
test_feats = np.array(test_feats, dtype=np.float32)

print(test_feats.shape)

(12, 25117)


In [184]:
print(test_feats.shape)

(12, 25117)


In [187]:
# apply PCA
# full_feats = np.append(train_feats, test_feats, axis=0)
# # mean = np.empty((0))
# mean, eigenvectors, eigenvalues = cv2.PCACompute2(full_feats, mean=None, )

# # print(full_feats.shape, eigenvectors.shape, eigenvalues)

kPCA = KernelPCA(n_components=10, kernel='rbf')
transformed_train_feats = kPCA.fit_transform(train_feats)

transformed_train_feats.shape

(42, 10)

In [188]:
# train knn classifier

knn = cv2.ml.KNearest_create()
knn.train(transformed_train_feats, cv2.ml.ROW_SAMPLE, labels)

True

In [189]:
# transform test features
transformed_test_feats = kPCA.transform(test_feats)
print(transformed_test_feats.shape)

(12, 10)


In [190]:
# test kNN
ret, results, neighbours ,dist = knn.findNearest(transformed_test_feats, 3)
print( "result:  {}\n".format(results) )
# print( "neighbours:  {}\n".format(neighbours) )
# print( "distance:  {}\n".format(dist) )

result:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

